In [97]:
import netCDF4
import numpy as np
import pandas as pd

## Create a netCDF4.Dataset object
- **`f`** is a `Dataset` object, representing an open netCDF file.
- printing the object gives you summary information, similar to *`ncdump -h`*.

In [98]:
f = netCDF4.Dataset('/Users/vsood/Downloads/prods_op_mogreps-uk_20140717_03_11_015.nc', 'r')
for cube in f.variables.keys():
    if len(f.variables[cube].shape) > 1:
        print(cube)

stratiform_snowfall_rate
cloud_base_altitude_assuming_only_consider_cloud_area_fraction_greater_than_2p5_oktas
cloud_area_fraction_assuming_maximum_random_overlap
wet_bulb_freezing_level_altitude
wet_bulb_potential_temperature
unknown
air_pressure_at_sea_level
air_temperature
air_temperature_0
time_1_bnds
forecast_period_1_bnds
air_temperature_1
air_temperature_2
dew_point_temperature
fog_area_fraction
geopotential_height
high_type_cloud_area_fraction
low_type_cloud_area_fraction
medium_type_cloud_area_fraction
relative_humidity
relative_humidity_0
specific_humidity
stratiform_rainfall_amount
forecast_period_2_bnds
time_2_bnds
stratiform_rainfall_rate
stratiform_snowfall_amount
surface_air_pressure
surface_downward_eastward_stress
surface_downward_northward_stress
surface_downwelling_longwave_flux
surface_downwelling_shortwave_flux_in_air
visibility_in_air
wind_speed_of_gust
x_wind
x_wind_0
x_wind_1
y_wind
y_wind_0
y_wind_1


In [99]:
air_visibility = f.variables['visibility_in_air']
wind_speed = f.variables['wind_speed_of_gust']
fog_area = f.variables['fog_area_fraction']
air_temperature = f.variables['air_temperature_2']
snowfall_rate = f.variables['stratiform_snowfall_rate']
snowfall_amount = f.variables['stratiform_snowfall_amount']
rainfall_rate = f.variables['stratiform_rainfall_rate']
rainfall_amount = f.variables['stratiform_rainfall_amount']
x_wind = f.variables['x_wind']
x_wind_0 = f.variables['x_wind_0']
x_wind_1 = f.variables['x_wind_1']
y_wind = f.variables['y_wind']
y_wind_0 = f.variables['y_wind_0']
y_wind_1 = f.variables['y_wind_1']

In [156]:
aviation_cubes = [air_visibility, wind_speed, fog_area, air_temperature, 
                 snowfall_rate,  rainfall_rate, x_wind, x_wind_0, x_wind_1, 
                 y_wind, y_wind_0,y_wind_1]

In [143]:
def create_arrays(dataset, dim_tuple, size):
    main_dim, *rest_of_dims = dim_tuple
    main_dim_data = dataset.variables[main_dim][:]
    for i in range(len(rest_of_dims)):
        rest_of_dims[i] = dataset.variables[rest_of_dims[i]][:]
    return np.vstack(np.meshgrid(main_dim_data, *rest_of_dims)).reshape(size, -1).T

In [157]:
for cube in aviation_cubes:
    size = cube.size
    dims = cube.dimensions
    print("cube: {}, size = {}, dims = {}".format(cube.name, size, dims))
    cube_data = cube[:].reshape(size,1) # reshaping all forecast data into a single columnß
    cube_data = create_arrays(f, dims, size)
    print(cube_data.size)
    print(cube_data.shape)
#     for dim in dims:
#         f.


cube: visibility_in_air, size = 692124, dims = ('time_0', 'grid_latitude', 'grid_longitude')
2076372
(3, 692124)
cube: wind_speed_of_gust, size = 692124, dims = ('time_1', 'grid_latitude', 'grid_longitude')
2076372
(3, 692124)
cube: fog_area_fraction, size = 692124, dims = ('time_0', 'grid_latitude', 'grid_longitude')
2076372
(3, 692124)
cube: air_temperature_2, size = 1384248, dims = ('time_0', 'pressure_0', 'grid_latitude', 'grid_longitude')
5536992
(4, 1384248)
cube: stratiform_snowfall_rate, size = 8305488, dims = ('time', 'grid_latitude', 'grid_longitude')
24916464
(3, 8305488)
cube: stratiform_rainfall_rate, size = 8305488, dims = ('time', 'grid_latitude', 'grid_longitude')
24916464
(3, 8305488)
cube: x_wind, size = 690861, dims = ('time_0', 'grid_latitude_0', 'grid_longitude_0')
2072583
(3, 690861)
cube: x_wind_0, size = 1381722, dims = ('time_0', 'pressure_0', 'grid_latitude_0', 'grid_longitude_0')
5526888
(4, 1381722)
cube: x_wind_1, size = 692124, dims = ('time_0', 'grid_lati

## We can reshape the array into a single column

In [ ]:
reshaped_air_temp = air_temperature[:].reshape(692124,1)

In [ ]:
time = f.variables['time_0'][:]
print(type(time))
lat = f.variables['grid_latitude'][:]
long = f.variables['grid_longitude'][:]

In [ ]:
new_array = np.vstack(np.meshgrid(time,lat,long)).reshape(3, -1).T

In [ ]:
df = pd.concat([pd.DataFrame(new_array), pd.DataFrame(reshaped_air_temp) - 273.15],axis=1)
df.to_csv('/Users/vsood/Downloads/new_array.csv')